# Titanic
#### El 15 de Abril del año 1912 lastimosamente muere 1502 personas del 2224 en el hundimiento del titanic.
#### Para este reto nos planteamos la siguiente pregunta, ¿Cuántas personas tenían más probabilidades de sobrevivir tomando en cuenta sus datos personales?
    Tales como:
        El nombre,
        La edad,
        El genero,
        La clase socioeconómico,
        etc.
#### Para este reto debemos de programar un modelo para respoder la pregunta.

In [1]:
#importamos las librerías a utilizar

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [2]:
#cargar los datos

train_data = pd.read_csv("./train.csv")
test_data = pd.read_csv("./test.csv")

In [3]:
#explorar los datos de entrenamiento

print(train_data.shape)
print(train_data.info())
print(pd.isnull(train_data).sum())
print(train_data.describe(include="all"))
train_data.head()

(891, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
d

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
#explorar los datos de prueba

print(test_data.shape)
print(test_data.info())
print(pd.isnull(test_data).sum())
print(test_data.describe(include="all"))
test_data.head()

(418, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB
None
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64
        PassengerId      Pclass                  

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
##Preprocesamiento de datos
#Cambiar etiquetas a valor

train_data['Sex'].replace(to_replace=["female","male"],value=[0,1],inplace=True)
train_data['Embarked'].replace(to_replace=["Q","S","C"],value=[0,1,2],inplace=True)

test_data['Sex'].replace(to_replace=["female","male"],value=[0,1],inplace=True)
test_data['Embarked'].replace(to_replace=["Q","S","C"],value=[0,1,2],inplace=True)

In [6]:
##Preprocesamiento de datos
#Rellenar campo de edad con el promedio de los campos con valores

train_age_mean = np.around(train_data['Age'].mean())
test_age_mean = np.around(test_data['Age'].mean())

train_data['Age'].replace(to_replace=np.nan, value=train_age_mean, inplace=True)
test_data['Age'].replace(to_replace=np.nan, value=test_age_mean, inplace=True)

In [7]:
##Preprocesamiento de datos
#Rellenar campo de edad con un rango de los campos con valores

range_edad = [0, 8, 15, 18, 25, 40, 60, 100]
valor_edad = [1, 2, 3, 4, 5, 6, 7]

train_data['Age'] = pd.cut(x=train_data['Age'], bins=range_edad, labels=valor_edad)
test_data['Age'] = pd.cut(x=test_data['Age'], bins=range_edad, labels=valor_edad)

In [8]:
##Preprocesamiento de datos
#Eliminar la columna Cabin debido que tiene muchos datos perdidos

train_data.drop(labels=['Cabin'], axis=1, inplace=True)
test_data.drop(labels=['Cabin'], axis=1, inplace=True)

In [9]:
##Preprocesamiento de datos
#Eliminar columnas que no aporta peso para el modelo

train_data.drop(labels=['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
test_data.drop(labels=['Name', 'Ticket'], axis=1, inplace=True)

In [10]:
##Preprocesamiento de datos
#Eliminar filas que tiene al menos un campo vacio

train_data.dropna(axis=0, how='any', inplace=True)
test_data.dropna(axis=0, how='any', inplace=True)

In [11]:
#verificacion de datos

print(train_data.shape)
print(test_data.shape)

(889, 8)
(417, 8)


In [12]:
##Aplicar modelo
#Construir columnas para la info de sobrevivientes

X = np.array(train_data.drop(['Survived'], 1))
y = np.array(train_data['Survived'])

In [13]:
##Aplicar modelo
#separacion de datos de entrenamineto y prueba

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [22]:
##Aplicar modelo
#regresion logistica

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
Y_pred = logreg.predict(X_test)
#Presicion del algoritmo
print(logreg.score(X_train, y_train))

0.8059071729957806


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [15]:
##Aplicar modelo
#Support Vector Machine

svc = SVC()
svc.fit(X_train, y_train)
Y_pred = svc.predict(X_test)
#Presicion del algoritmo
print(svc.score(X_train, y_train))

0.679324894514768


In [16]:
##Aplicar modelo
#K neighbors

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
Y_pred = knn.predict(X_test)
#Presicion del algoritmo
print(knn.score(X_train, y_train))

0.8593530239099859


In [17]:
##Prediccion
#Regresion logistica

ids = test_data['PassengerId']

prediccion_logreg = logreg.predict(test_data.drop('PassengerId', axis=1))
out_logreg = pd.DataFrame({ 'PassengerId' : ids, 'Survived': prediccion_logreg })

out_logreg['Survived'].value_counts()

0    267
1    150
Name: Survived, dtype: int64

In [18]:
##Prediccion
#Regresion logistica

ids = test_data['PassengerId']

prediccion_svc = svc.predict(test_data.drop('PassengerId', axis=1))
out_svc = pd.DataFrame({ 'PassengerId' : ids, 'Survived': prediccion_svc })

out_svc['Survived'].value_counts()

0    335
1     82
Name: Survived, dtype: int64

In [19]:
##Prediccion
#Regresion logistica

ids = test_data['PassengerId']

prediccion_knn = knn.predict(test_data.drop('PassengerId', axis=1))
out_knn = pd.DataFrame({ 'PassengerId' : ids, 'Survived': prediccion_knn })

out_knn['Survived'].value_counts()

0    249
1    168
Name: Survived, dtype: int64